# **Cluster South Mumbai**

# **Version 10** 

# **Get required Libraries**

In [2]:
import pandas as pd
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import numpy as np

from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns
from urllib.request import urlopen

import requests
# import k-means from clustering stage
from sklearn.cluster import KMeans

print("Library imported 61.")


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Library imported 61.


In [3]:
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print("Library imported 62.")


Library imported 62.


## **Mumbai Neighborhoods.**

In [0]:
areas_south = ["Colaba","Apollo Bandar","Fort","Churchgate","Nariman Point" \
        ,"Marine Lines","Walkeshwar","Malabar Hill","Kalbadevi","Bhuleshwar" \
        ,"Masjid Bandar","Darukhana","Pydhoni","Nagpada","Dongri" \
        ,"Byculla","Grand Road","Mazgaon","Mumbai Central","Mahalaxmi" \
        ,"Worli"]

areas_west = ["Dadar","Bandra West","Juhu","Andheri","Kalina" \
              ,"Santa Cruz","Mahim","Khar"]

areas_east = ["Kurla","Ghatkopar","Chembur","Govandi","Mulund West"] 
areas_north = ["Powai","Jogeshwari","Malad","Borivali","Mira Road"] 

#boroughs
boroughs_south_mumbai = ["South Mumbai","South Mumbai","South Mumbai","South Mumbai","South Mumbai", \
             "South Mumbai","South Mumbai","South Mumbai","South Mumbai","South Mumbai", \
             "South Mumbai","South Mumbai","South Mumbai","South Mumbai","South Mumbai", \
             "South Mumbai","South Mumbai","South Mumbai","South Mumbai","South Mumbai", \
             "South Mumbai"]

boroughs_west_mumbai = ["West Mumbai","West Mumbai","West Mumbai","West Mumbai","West Mumbai" \
             ,"West Mumbai","West Mumbai","West Mumbai"]

boroughs_east_mumbai = ["East Mumbai","East Mumbai","East Mumbai","East Mumbai","East Mumbai"]

boroughs_north_mumbai = ["North Mumbai","North Mumbai","North Mumbai","North Mumbai","North Mumbai"]

#### **Note : This split was done due to limitations faced during call to API for retrieving the coordinates of all 39 neighborhoods.**

In [5]:
boroughs1 =  boroughs_south_mumbai
boroughs2 =  boroughs_west_mumbai + boroughs_east_mumbai + boroughs_north_mumbai

areas1 = areas_south
areas2 = areas_west + areas_east + areas_north

#Building full address to pass to API
full_address1 = []
for area in areas1:
  full_address1.append(area + ", Mumbai")

full_address2 = []
for area in areas2:
  full_address2.append(area + ", Mumbai") 

print("Lenght of areas1={} , boroughs1 = {} , full_address1 = {}" \
.format(len(areas1),len(boroughs1),len(full_address1)))

print("Lenght of areas2={} , boroughs2 = {} , full_address2 = {}" \
.format(len(areas2),len(boroughs2),len(full_address2))) 

#Create Dictionary first
dic_area1 = {"Neighborhood": areas1,"Boroughs": boroughs1,"Address":full_address1}
dic_area2 = {"Neighborhood": areas2,"Boroughs": boroughs2,"Address":full_address2}
print("dic_area1:",dic_area1)
print("dic_area2:",dic_area2)

#Create dataframes from dictionary
df_area1 = pd.DataFrame.from_dict(dic_area1)
df_area2 = pd.DataFrame.from_dict(dic_area2)

print("df_area1:",df_area1.shape)
print("df_area2:",df_area2.shape)



Lenght of areas1=21 , boroughs1 = 21 , full_address1 = 21
Lenght of areas2=18 , boroughs2 = 18 , full_address2 = 18
dic_area1: {'Neighborhood': ['Colaba', 'Apollo Bandar', 'Fort', 'Churchgate', 'Nariman Point', 'Marine Lines', 'Walkeshwar', 'Malabar Hill', 'Kalbadevi', 'Bhuleshwar', 'Masjid Bandar', 'Darukhana', 'Pydhoni', 'Nagpada', 'Dongri', 'Byculla', 'Grand Road', 'Mazgaon', 'Mumbai Central', 'Mahalaxmi', 'Worli'], 'Boroughs': ['South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai', 'South Mumbai'], 'Address': ['Colaba, Mumbai', 'Apollo Bandar, Mumbai', 'Fort, Mumbai', 'Churchgate, Mumbai', 'Nariman Point, Mumbai', 'Marine Lines, Mumbai', 'Walkeshwar, Mumbai', 'Malabar Hill, Mumbai', 'Kalbadevi, Mumbai', 'Bhuleshwar, Mumba

### **Get coordinates of each neighborhood of Mumbai.**

##### **Let's pass South Mumbai dataframe :  df_area1**

##### **Use geopy library to get the latitude and longitude values of Mumbai neigborhoods**
####**In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.**

In [7]:
# South Mumbai first

df2 = df_area1
df2.head()

data = []
for  neighborhood,address,borough in zip(df2['Neighborhood'], df2['Address'],df2['Boroughs']):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    data.append([neighborhood,borough,latitude,longitude])

col_names =  ['Neighborhood', 'Borough', 'Latitude','Longitude']
neighborhoods = pd.DataFrame(data,columns=col_names)

# Assign to neighborhoods1 : South Mumbai
neighborhoods1 = neighborhoods
print("Coordiates for {} South Mumbai neighborhoods obtained.".format(neighborhoods1.shape[0]) )
neighborhoods1.tail()


Coordiates for 21 South Mumbai neighborhoods obtained.


,Neighborhood,Borough,Latitude,Longitude
16,Grand Road,South Mumbai,18.938771,72.835335
17,Mazgaon,South Mumbai,18.968052,72.840012
18,Mumbai Central,South Mumbai,18.969586,72.819315
19,Mahalaxmi,South Mumbai,18.982568,72.824160
20,Worli,South Mumbai,19.011696,72.818070


### **Now let's pass east+west+north Mumabi : df_area2**

In [0]:
df2 = df_area2
df2.tail()

,Neighborhood,Boroughs,Address
13,Powai,North Mumbai,"Powai, Mumbai"
14,Jogeshwari,North Mumbai,"Jogeshwari, Mumbai"
15,Malad,North Mumbai,"Malad, Mumbai"
16,Borivali,North Mumbai,"Borivali, Mumbai"
17,Mira Road,North Mumbai,"Mira Road, Mumbai"


In [8]:
# Now second iteration for Rest of Mumbai.

df2 = df_area2
df2.tail()

data = []
for  neighborhood,address,borough in zip(df2['Neighborhood'], df2['Address'],df2['Boroughs']):
  print("neighborhood:", neighborhood)
  print("address:", address)
  print("borough:", borough)
  geolocator = Nominatim(user_agent="ny_explorer")
  location = geolocator.geocode(address)
  latitude = location.latitude
  longitude = location.longitude
  print("latitude:", latitude)
  print("longitude:", longitude)
  data.append([neighborhood,borough,latitude,longitude])

col_names =  ['Neighborhood', 'Borough', 'Latitude','Longitude']
neighborhoods = pd.DataFrame(data,columns=col_names)

#Assign to neighborhoods2 : Rest of Mumbai
neighborhoods2 = neighborhoods
print("Coordiates for {} Rest of Mumbai neighborhoods obtained.".format(neighborhoods2.shape[0]) )
neighborhoods2.tail()

neighborhood: Dadar
address: Dadar, Mumbai
borough: West Mumbai
latitude: 19.019282
longitude: 72.8428757
neighborhood: Bandra West
address: Bandra West, Mumbai
borough: West Mumbai
latitude: 19.0583358
longitude: 72.8302669
neighborhood: Juhu
address: Juhu, Mumbai
borough: West Mumbai
latitude: 19.1070215
longitude: 72.8275275
neighborhood: Andheri
address: Andheri, Mumbai
borough: West Mumbai
latitude: 19.1196976
longitude: 72.8464205
neighborhood: Kalina
address: Kalina, Mumbai
borough: West Mumbai
latitude: 19.079273
longitude: 72.8612672
neighborhood: Santa Cruz
address: Santa Cruz, Mumbai
borough: West Mumbai
latitude: 19.0793694
longitude: 72.8470855
neighborhood: Mahim
address: Mahim, Mumbai
borough: West Mumbai
latitude: 19.0423145
longitude: 72.8398344
neighborhood: Khar
address: Khar, Mumbai
borough: West Mumbai
latitude: 19.0696584
longitude: 72.8398944
neighborhood: Kurla
address: Kurla, Mumbai
borough: East Mumbai
latitude: 19.0652797
longitude: 72.8793805
neighborhood: G

,Neighborhood,Borough,Latitude,Longitude
13,Powai,North Mumbai,19.118720,72.907348
14,Jogeshwari,North Mumbai,19.134899,72.848820
15,Malad,North Mumbai,19.186719,72.848588
16,Borivali,North Mumbai,19.229068,72.857363
17,Mira Road,North Mumbai,19.187896,72.836596


## **Now Let's concat two dataframes for clubbing entire Mumbai dataframe to one.** 

In [9]:
neighborhoods = neighborhoods1.append(neighborhoods2,ignore_index=True)
neighborhoods = neighborhoods.reset_index()
print("Merged data frame have {} rows.".format(neighborhoods.shape[0]) )
neighborhoods.tail()


Merged data frame have 39 rows.


,index,Neighborhood,Borough,Latitude,Longitude
34,34,Powai,North Mumbai,19.118720,72.907348
35,35,Jogeshwari,North Mumbai,19.134899,72.848820
36,36,Malad,North Mumbai,19.186719,72.848588
37,37,Borivali,North Mumbai,19.229068,72.857363
38,38,Mira Road,North Mumbai,19.187896,72.836596


#### **Let's get coordinates of Mumbai.**

In [10]:
address = "Mumbai"
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
mumbai_latitude = latitude
mumbai_longitude = longitude
print ("Latitude={} and Longitude={}".format(mumbai_latitude,mumbai_longitude))

Latitude=18.9387711 and Longitude=72.8353355


### **Create a map of Mumbai with neighborhoods superimposed on top.**


 Here folium library is being used to create city map.The add masrkers to show neighborhoods of Mumbai.

In [11]:
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

### **Four Square API to Get Venues** 

##### Four Square API are being used here to fetch the venues around a location. It needs latitude and longitude of location and returns venues along with categories.

##### **Define Foursquare Credentials and Version**

In [12]:
CLIENT_ID = 'AED1YJAEJO1BSJVJJ5KT2Y2BDGDVPCB21WUXERHTA4B4XXKC' # Foursquare ID
CLIENT_SECRET = '5WF0PKUAGWG30ASKZSBV12XHC2IDIQLEHRRPU541XI2JVQCH' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


credentails:
CLIENT_ID: AED1YJAEJO1BSJVJJ5KT2Y2BDGDVPCB21WUXERHTA4B4XXKC
CLIENT_SECRET:5WF0PKUAGWG30ASKZSBV12XHC2IDIQLEHRRPU541XI2JVQCH


#### **Let's create a function to get nearby venues of each neighborhoods in Mumbai.**

##### Tried to keep an optimum value of radius. With more radius it fetches more vanues but chances of overlapping of neighborhood increases as well.

In [13]:
radius=650
LIMIT = 100
print("The radius is = {} and LIMIT is = {}".format(radius,LIMIT))

def getNearbyVenues(names, latitudes, longitudes, radius=650):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


The radius is = 650 and LIMIT is = 100


##### **Running the above function on each neighborhood and create a new dataframe.**

In [14]:
south_mumbai_data = neighborhoods
south_mumbai_venues = getNearbyVenues(names=south_mumbai_data['Neighborhood'],
                                   latitudes=south_mumbai_data['Latitude'],
                                   longitudes=south_mumbai_data['Longitude']
                                   
                                  )
print("dtypes:",south_mumbai_venues.dtypes)
print("Total venues retrieved:",south_mumbai_venues.shape[0])


Colaba
Apollo Bandar
Fort
Churchgate
Nariman Point
Marine Lines
Walkeshwar
Malabar Hill
Kalbadevi
Bhuleshwar
Masjid Bandar
Darukhana
Pydhoni
Nagpada
Dongri
Byculla
Grand Road
Mazgaon
Mumbai Central
Mahalaxmi
Worli
Dadar
Bandra West
Juhu
Andheri
Kalina
Santa Cruz
Mahim
Khar
Kurla
Ghatkopar
Chembur
Govandi
Mulund West
Powai
Jogeshwari
Malad
Borivali
Mira Road
dtypes: Neighborhood               object
Neighborhood Latitude     float64
Neighborhood Longitude    float64
Venue                      object
Venue Latitude            float64
Venue Longitude           float64
Venue Category             object
dtype: object
Total venues retrieved: 1118


#### **Understand the venue categories well.**

In [0]:
south_mumbai_venues.head(2)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Colaba,18.915091,72.825969,Charagh Din,18.915254,72.824151,Men's Store
1,Colaba,18.915091,72.825969,IMBISS Meating Joint,18.917157,72.827018,German Restaurant


In [0]:
# backup
south_mumbai_venues_unclean = south_mumbai_venues
# save as csv file
south_mumbai_venues_unclean = south_mumbai_venues_unclean.to_csv("south_mumbai_venues_unclean")

In [16]:
# merge with master table

south_mumbai_venues_merge_1 = pd.merge(south_mumbai_venues,neighborhoods,on="Neighborhood", how="inner")
south_mumbai_venues_merge_1 = south_mumbai_venues_merge_1[["Neighborhood","Borough", "Neighborhood Latitude" \
                               , "Neighborhood Longitude", "Venue", "Venue Latitude" \
                               , "Venue Longitude", "Venue Category" \
                                ]]
south_mumbai_venues_merge_1.head(1)


,Neighborhood,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Colaba,South Mumbai,18.915091,72.825969,Charagh Din,18.915254,72.824151,Men's Store


# **Data Cleansing** 

##### Identify the issues with data and rectify these errors like duplicate data etc. in order to make data fit for accurate analysis.

#### **Drop Duplicates Venues**

##### South Mumbai is a narrow strip, so chances of overlap are possible. Therefore, duplicates venues needs to be discarded.

In [17]:
print("rows before dropping duplicates={}".format(south_mumbai_venues.shape[0]))
south_mumbai_venues = south_mumbai_venues.drop_duplicates(subset=['Venue','Venue Latitude', 'Venue Longitude'])
print("rows after dropping duplicates={}".format(south_mumbai_venues.shape[0]))


rows before dropping duplicates=1118
rows after dropping duplicates=1003


### **Ignore rows with Indian Restaurants**.

##### Indian restaurants are very common and in large number in entire Mumbai. Therefore, decided to get rid of Indian Restaurants by using filter.

In [18]:
print("rows bef filter:",south_mumbai_venues.shape[0])
south_mumbai_venues = south_mumbai_venues[(south_mumbai_venues['Venue Category'] != "Indian Restaurant")].reset_index()
print("rows aft filter:",south_mumbai_venues.shape[0])




rows bef filter: 1003
rows aft filter: 868


In [0]:
# backup
south_mumbai_venues_minus_ind_rest = south_mumbai_venues

### **Group by Category**

In [20]:
south_mumbai_venues_grp_ven_cat_1 = south_mumbai_venues_merge_1.groupby("Venue Category").count()
south_mumbai_venues_grp_ven_cat_2 = south_mumbai_venues_grp_ven_cat_1.sort_values("Neighborhood" , ascending=False)
south_mumbai_venues_grp_ven_cat_2 = south_mumbai_venues_grp_ven_cat_2.rename({"Neighborhood":"Count"},axis=1)
south_mumbai_venues_grp_ven_cat_2 = south_mumbai_venues_grp_ven_cat_2[["Count"]]
south_mumbai_venues_grp_ven_cat_2.head(1)

#df2 = df.rename({'a': 'X', 'b': 'Y'}, axis=1) 

,Count
Venue Category,
Indian Restaurant,164


In [21]:
south_mumbai_venues_grp_ven_cat_2.head(10)



,Count
Venue Category,
Indian Restaurant,164
Café,61
Fast Food Restaurant,50
Chinese Restaurant,41
Coffee Shop,40
Bar,38
Restaurant,34
Hotel,32
Dessert Shop,29


In [0]:
# save as csv
south_mumbai_venues_grp_ven_cat_2.to_csv("south_mumbai_venues_grp_ven_cat_2.csv")
# backup
south_mumbai_venues_grp_ven_cat_2_bkp = south_mumbai_venues_grp_ven_cat_2


### **Group by Neighborhood**

In [23]:
# group by neighborhoods
south_mumbai_venues_grp_nbhood_1 = south_mumbai_venues_merge_1.groupby("Neighborhood").count()
south_mumbai_venues_grp_nbhood_1.head(1)
south_mumbai_venues_grp_nbhood_1 = south_mumbai_venues_grp_nbhood_1.rename({"Borough" : "Count"}, axis=1)
south_mumbai_venues_grp_nbhood_2 = south_mumbai_venues_grp_nbhood_1.sort_values("Count" , ascending=False)
south_mumbai_venues_grp_nbhood_2 = south_mumbai_venues_grp_nbhood_2[["Count"]]
south_mumbai_venues_grp_nbhood_2.head()


,Count
Neighborhood,
Bandra West,100
Fort,79
Apollo Bandar,74
Juhu,70
Powai,69


In [24]:
# Display Top 10 Neighborhoods with maximum count of venues. 
south_mumbai_venues_grp_nbhood_2.head(10)

,Count
Neighborhood,
Bandra West,100
Fort,79
Apollo Bandar,74
Juhu,70
Powai,69
Churchgate,66
Nariman Point,46
Mira Road,45
Dadar,37


In [0]:
# save as csv
south_mumbai_venues_grp_nbhood_2.to_csv("south_mumbai_venues_grp_nbhood_2.csv")
south_mumbai_venues_grp_nbhood_2_bkp = south_mumbai_venues_grp_nbhood_2
 

In [26]:
# bottom 10 
south_mumbai_venues_grp_nbhood_2 = south_mumbai_venues_grp_nbhood_2.sort_values("Count" , ascending=True)
south_mumbai_venues_grp_nbhood_2.head(10)

,Count
Neighborhood,
Darukhana,2
Jogeshwari,6
Dongri,6
Govandi,6
Kurla,8
Kalina,9
Malad,9
Byculla,10
Mahim,10


#### **Unique Categories**

In [27]:
print('There are {} uniques categories.'.format(len(south_mumbai_venues['Venue Category'].unique())))


There are 158 uniques categories.


### **Clubbing Similar Categories**

####For example garden and parks can be clubbed together to gardens/parks etc.

In [28]:
south_mumbai_venues.replace(to_replace =['Movie Theater'],value = 'Theater\Multiplex', inplace=True)
south_mumbai_venues.replace(to_replace =['Theater'],value = 'Theater\Multiplex' , inplace=True)
south_mumbai_venues.replace(to_replace =['Multiplex'],value = 'Theater\Multiplex' , inplace=True)
south_mumbai_venues.replace(to_replace =['Amphitheater'],value = 'Theater\Multiplex' , inplace=True)
south_mumbai_venues.replace(to_replace =['Indie Movie Theater'],value = 'Theater\Multiplex' , inplace=True)
south_mumbai_venues.replace(to_replace =['Performing Arts Venue'],value = 'Theater\Multiplex' , inplace=True)

south_mumbai_venues.replace(to_replace =['Garden','Park'],value = 'Park/Garden', inplace=True)

south_mumbai_venues.replace(to_replace =['Pub'],value = 'Bar/Pub', inplace=True )
south_mumbai_venues.replace(to_replace =['Gastropub'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Bar'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Cocktail Bar'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Wine Bar'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Hotel Bar'],value = 'Bar/Pub', inplace=True)
south_mumbai_venues.replace(to_replace =['Sports Bar'],value = 'Bar/Pub', inplace=True)

south_mumbai_venues.replace(to_replace =['Train Station' \
                                         ,'Bus Station'
                                         ,'Metro Station'
                                         ,'Train'
],value = 'Train/Metro/Bus', inplace=True )

south_mumbai_venues.replace(to_replace =['Spa' \
                                         ,'Gym'
                                         ,'Gym / Fitness Center'
                                         ,'Yoga Studio'
                                         ,'Track'
                                         ,'Massage Studio'
],value = 'Fitness/Gym/Spa', inplace=True )

south_mumbai_venues.replace(to_replace =['Tennis Court' \
                                         ,'Athletics & Sports'
                                         ,'Cricket Ground'
                                         ,'Hockey Arena'
                                         ,'Field'
                                         ,'Stadium'
                                         ,'Soccer Field'
                                         ,'Playground'
],value = 'Sports', inplace=True )

south_mumbai_venues.shape

(868, 8)

### **Check neighborhoods with very less venues**

In [31]:
south_mumbai_venues_group_nbhood = south_mumbai_venues
south_mumbai_venues_group_nbhood = south_mumbai_venues_group_nbhood.groupby(['Neighborhood']) \
                             .count() 
south_mumbai_venues_group_nbhood = south_mumbai_venues_group_nbhood.sort_values(['index'], ascending=True)
south_mumbai_venues_group_nbhood.head(2)

,index,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Darukhana,2,2,2,2,2,2,2
Dongri,2,2,2,2,2,2,2


### **Drop Neigborhoods**

### **Darukhana**

##### Drop some neiborhoods with very less venues.

In [32]:
print("before filter:",south_mumbai_venues.shape)
south_mumbai_venues = south_mumbai_venues[south_mumbai_venues.Neighborhood != 'Darukhana']
print("after filter:",south_mumbai_venues.shape)

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Darukhana']
print("aft:",south_mumbai_data.shape)


before filter: (868, 8)
after filter: (866, 8)
bef: (39, 5)
aft: (38, 5)


### **Merge Neighborhoods**

##### Merge Neigborhoods with very less venues to nearby venues. 

In [40]:
# Get coordinates of Nagpada

Nagpada = south_mumbai_venues
Nagpada = Nagpada[Nagpada.Neighborhood == 'Nagpada' ].head(1)
nagpada_latitude = Nagpada['Neighborhood Latitude'] 
nagpada_latitude = nagpada_latitude.iloc[0]
nagpada_longitude = Nagpada['Neighborhood Longitude'] 
nagpada_longitude = nagpada_longitude.iloc[0]
print("Nagpada latitude={} and longitude={}".format(nagpada_latitude,nagpada_longitude))


Nagpada latitude=18.9681782 and longitude=72.8286009


### **Merge Pydhoni to Nagpada**

In [41]:
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Pydhoni', 'Neighborhood Latitude'] = nagpada_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Pydhoni', 'Neighborhood Longitude'] = nagpada_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Pydhoni', 'Neighborhood'] = 'Nagpada'
south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Pydhoni']
south_mumbai_venues[south_mumbai_venues.Venue == 'Kanji Manji Kothari']
#south_mumbai_venues.iloc[313]

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Pydhoni']
print("aft:",south_mumbai_data.shape)


bef: (34, 5)
aft: (34, 5)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### **Merge Dongri to Nagpada**

In [42]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Dongri']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Dongri', 'Neighborhood Latitude'] = nagpada_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Dongri', 'Neighborhood Longitude'] = nagpada_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Dongri', 'Neighborhood'] = 'Nagpada'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Dongri']
south_mumbai_venues.iloc[332:334]

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Dongri']
print("aft:",south_mumbai_data.shape)

bef: (34, 5)
aft: (34, 5)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road']
#south_mumbai_venues.iloc[340:344]

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
341,487,Grand Road,18.938771,72.835335,Chhatrapati Shivaji Maharaj Terminus,18.940088,72.835257,Train/Metro/Bus
342,507,Grand Road,18.938771,72.835335,Press Club Mumbai,18.940721,72.832469,Bar/Pub
343,512,Grand Road,18.938771,72.835335,Model Cafe,18.936027,72.840418,Café
344,513,Grand Road,18.938771,72.835335,grand hotel,18.936234,72.840724,Hotel


In [43]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road', 'Neighborhood Latitude'] = nagpada_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road', 'Neighborhood Longitude'] = nagpada_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Grand Road', 'Neighborhood'] = 'Nagpada'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Grand Road']
south_mumbai_venues.iloc[340:344]

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Grand Road']
print("aft:",south_mumbai_data.shape)

bef: (34, 5)
aft: (34, 5)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla']
south_mumbai_venues.iloc[334:340]

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
335,474,Byculla,18.976622,72.832794,Bhau Daji Lad Museum,18.979140,72.834449,History Museum
336,476,Byculla,18.976622,72.832794,Cafe Renault,18.974986,72.833758,Bar/Pub
337,477,Byculla,18.976622,72.832794,Gloria Restaurant,18.975485,72.833884,Asian Restaurant
338,479,Byculla,18.976622,72.832794,Domino's Pizza,18.978071,72.838701,Pizza Place
339,480,Byculla,18.976622,72.832794,Jijamata Udyan - Ranibaug,18.978606,72.834980,Zoo
340,481,Byculla,18.976622,72.832794,Byculla Bakery,18.977695,72.834005,Bakery


In [44]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla', 'Neighborhood Latitude'] = nagpada_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla', 'Neighborhood Longitude'] = nagpada_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Byculla', 'Neighborhood'] = 'Nagpada'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Byculla']
south_mumbai_venues.iloc[334:340]

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Byculla']
print("aft:",south_mumbai_data.shape)

bef: (34, 5)
aft: (34, 5)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### **Merge Govandi with Chembur**

In [45]:
# Get coordinates of Chembur

Chembur = south_mumbai_venues
Chembur = Chembur[Chembur.Neighborhood == 'Chembur' ].head(1)
Chembur_latitude = Chembur['Neighborhood Latitude'] 
Chembur_latitude = Chembur_latitude.iloc[0]
Chembur_longitude = Chembur['Neighborhood Longitude'] 
Chembur_longitude = Chembur_longitude.iloc[0]
print("Chembur latitude={} and longitude={}".format(Chembur_latitude,Chembur_longitude))
Chembur[Chembur.Neighborhood == 'Chembur' ].head(1)

Chembur latitude=19.0612128 and longitude=72.8975909


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
686,910,Chembur,19.061213,72.897591,Le Café,19.061791,72.899479,Café


In [0]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi']
south_mumbai_venues.iloc[688:693]

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
689,925,Govandi,19.055369,72.91507,Govandi Station,19.054849,72.915618,Platform
690,926,Govandi,19.055369,72.91507,Gavand Vada Pao center,19.055100,72.914816,Hot Dog Joint
691,927,Govandi,19.055369,72.91507,Natraj Store,19.055045,72.915612,Food & Drink Shop
692,928,Govandi,19.055369,72.91507,Munna Sandwich,19.055237,72.914551,Food Truck
693,931,Govandi,19.055369,72.91507,Deonar Park,19.051004,72.914629,Park


In [46]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi', 'Neighborhood Latitude'] = Chembur_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi', 'Neighborhood Longitude'] = Chembur_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Govandi', 'Neighborhood'] = 'Chembur'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Govandi']
south_mumbai_venues.iloc[688:693]

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Govandi']
print("aft:",south_mumbai_data.shape)

bef: (34, 5)
aft: (33, 5)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### **Merge Kalina with Kurla**

In [47]:
# Get coordinates of Kurla

Kurla = south_mumbai_venues
Kurla = Kurla[Kurla.Neighborhood == 'Kurla' ].head(1)
Kurla_latitude = Kurla['Neighborhood Latitude'] 
Kurla_latitude = Kurla_latitude.iloc[0]
Kurla_longitude = Kurla['Neighborhood Longitude'] 
Kurla_longitude = Kurla_longitude.iloc[0]
print("Kurla latitude={} and longitude={}".format(Kurla_latitude,Kurla_longitude))
Kurla[Kurla.Neighborhood == 'Kurla' ].head(1)

Kurla latitude=19.0652797 and longitude=72.8793805


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
662,880,Kurla,19.06528,72.87938,Swastik Farsan Mart,19.065989,72.878838,Grocery Store


In [0]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina']
south_mumbai_venues.iloc[588:594]

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
589,798,Kalina,19.079273,72.861267,King Chilly,19.078382,72.866350,Chinese Restaurant
590,799,Kalina,19.079273,72.861267,The Camp,19.077917,72.865643,Asian Restaurant
591,800,Kalina,19.079273,72.861267,Sadichha,19.081684,72.855762,Seafood Restaurant
592,801,Kalina,19.079273,72.861267,SONY CENTRE,19.077760,72.864600,Electronics Store
593,802,Kalina,19.079273,72.861267,king chilli kalina,19.079660,72.865732,Chinese Restaurant
594,803,Kalina,19.079273,72.861267,Imbiss Meating joint,19.078304,72.865934,Burger Joint


In [48]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina', 'Neighborhood Latitude'] = Kurla_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina', 'Neighborhood Longitude'] = Kurla_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Kalina', 'Neighborhood'] = 'Kurla'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Kalina']
south_mumbai_venues.iloc[588:594]

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Kalina']
print("aft:",south_mumbai_data.shape)



bef: (33, 5)
aft: (32, 5)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### **Merge Malad with Andheri**

In [49]:
# Get coordinates of Andheri

Andheri = south_mumbai_venues
Andheri = Andheri[Andheri.Neighborhood == 'Andheri' ].head(1)
Andheri_latitude = Andheri['Neighborhood Latitude'] 
Andheri_latitude = Andheri_latitude.iloc[0]
Andheri_longitude = Andheri['Neighborhood Longitude'] 
Andheri_longitude = Andheri_longitude.iloc[0]
print("Chembur latitude={} and longitude={}".format(Andheri_latitude,Andheri_longitude))
Andheri[Andheri.Neighborhood == 'Andheri' ].head(1)

Chembur latitude=19.1196976 and longitude=72.8464205


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
595,797,Andheri,19.119698,72.84642,Merwans Cake shop,19.1193,72.845418,Bakery


In [0]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad']
south_mumbai_venues.iloc[773:780]

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
774,1024,Malad,19.186719,72.848588,Gupta's Bhel and Pani Puri House,19.184527,72.845946,Snack Place
775,1025,Malad,19.186719,72.848588,MM Mithaiwala,19.187323,72.848164,Dessert Shop
776,1026,Malad,19.186719,72.848588,Aditi,19.185180,72.846718,Fast Food Restaurant
777,1027,Malad,19.186719,72.848588,Malad Railway Station,19.187100,72.848678,Train/Metro/Bus
778,1028,Malad,19.186719,72.848588,platform1,19.187324,72.849162,Tourist Information Center
779,1029,Malad,19.186719,72.848588,aditi fast food hotel,19.185317,72.846710,Fast Food Restaurant
780,1031,Malad,19.186719,72.848588,Sapna Veg. Restaurant,19.192203,72.846929,Fast Food Restaurant


In [50]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad', 'Neighborhood Latitude'] = Andheri_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad', 'Neighborhood Longitude'] = Andheri_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malad', 'Neighborhood'] = 'Andheri'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Malad']
south_mumbai_venues.iloc[773:780]

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Malad']
print("aft:",south_mumbai_data.shape)

bef: (32, 5)
aft: (31, 5)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### **Merge Mahim to Bandra**

In [51]:
# Get coordinates of Bandra West

Bandra = south_mumbai_venues
Bandra = Bandra[Bandra.Neighborhood == 'Bandra West' ].head(1)
Bandra_latitude = Bandra['Neighborhood Latitude'] 
Bandra_latitude = Bandra_latitude.iloc[0]
Bandra_longitude = Bandra['Neighborhood Longitude'] 
Bandra_longitude = Bandra_longitude.iloc[0]
print("Bandra latitude={} and longitude={}".format(Bandra_latitude,Bandra_longitude))
Bandra[Bandra.Neighborhood == 'Bandra West' ].head(1)

Bandra latitude=19.0583358 and longitude=72.8302669


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
442,627,Bandra West,19.058336,72.830267,Almeida Park,19.057656,72.831541,Park/Garden


In [0]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim']
south_mumbai_venues.iloc[610:618]

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
611,822,Mahim,19.042314,72.839834,Baba Falooda,19.039995,72.841259,Dessert Shop
612,823,Mahim,19.042314,72.839834,Cafe Irani Chai,19.039096,72.842720,Breakfast Spot
613,827,Mahim,19.042314,72.839834,Domino's Pizza,19.040252,72.841394,Pizza Place
614,828,Mahim,19.042314,72.839834,Tazza Fast Food,19.039052,72.841866,Fast Food Restaurant
615,829,Mahim,19.042314,72.839834,Quarter House,19.037115,72.841462,Bar/Pub
616,830,Mahim,19.042314,72.839834,Xavier Institute of Engineering,19.045061,72.841716,Arcade
617,831,Mahim,19.042314,72.839834,Rediff.com,19.042069,72.842838,Office
618,832,Mahim,19.042314,72.839834,Mahim Railway Station,19.043431,72.844425,Train/Metro/Bus


In [52]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim', 'Neighborhood Latitude'] = Bandra_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim', 'Neighborhood Longitude'] = Bandra_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Mahim', 'Neighborhood'] = 'Bandra West'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Mahim']
south_mumbai_venues.iloc[610:618]

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Mahim']
print("aft:",south_mumbai_data.shape)

bef: (31, 5)
aft: (30, 5)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### **Merge Malabar Hill to Walkeshwar**

In [53]:
# Get coordinates of Bandra West

Walkeshwar = south_mumbai_venues
Walkeshwar = Walkeshwar[Walkeshwar.Neighborhood == 'Walkeshwar' ].head(1)
Walkeshwar_latitude = Walkeshwar['Neighborhood Latitude'] 
Walkeshwar_latitude = Walkeshwar_latitude.iloc[0]
Walkeshwar_longitude = Walkeshwar['Neighborhood Longitude'] 
Walkeshwar_longitude = Walkeshwar_longitude.iloc[0]
print("Walkeshwar latitude={} and longitude={}".format(Walkeshwar_latitude,Walkeshwar_longitude))
Walkeshwar[Walkeshwar.Neighborhood == 'Walkeshwar' ].head(1)

Walkeshwar latitude=18.9553434 and longitude=72.8079469


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
262,328,Walkeshwar,18.955343,72.807947,Starbucks,18.956426,72.812065,Coffee Shop


In [0]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill']
south_mumbai_venues.iloc[270:280]

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
270,345,Walkeshwar,18.955343,72.807947,HDFC Bank,18.953883,72.802378,Bank
271,347,Malabar Hill,18.958162,72.803366,Priyadarshini Park,18.957515,72.799614,Park
272,348,Malabar Hill,18.958162,72.803366,Santosh Sagar,18.961177,72.801425,Fast Food Restaurant
273,350,Malabar Hill,18.958162,72.803366,Country Of Origin,18.955266,72.798743,Dessert Shop
274,352,Malabar Hill,18.958162,72.803366,Naturals Ice Cream,18.962432,72.801471,Dessert Shop
275,355,Malabar Hill,18.958162,72.803366,210°Veg Gourmet Bakery,18.961828,72.802137,Bakery
276,357,Malabar Hill,18.958162,72.803366,Tune Up,18.954427,72.803509,Fitness/Gym/Spa
277,358,Malabar Hill,18.958162,72.803366,Contempory Arts,18.963036,72.803092,Arts & Crafts Store
278,359,Malabar Hill,18.958162,72.803366,Tart,18.963942,72.803440,Bakery
279,362,Kalbadevi,18.949258,72.827938,Crawford Market,18.946761,72.831225,Market


In [54]:
#south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill']
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill', 'Neighborhood Latitude'] = Walkeshwar_latitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill', 'Neighborhood Longitude'] = Walkeshwar_longitude
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Malabar Hill', 'Neighborhood'] = 'Walkeshwar'
#south_mumbai_venues[south_mumbai_venues.Neighborhood == 'Malabar Hill']
south_mumbai_venues.iloc[270:280]

print("bef:",south_mumbai_data.shape)
south_mumbai_data = south_mumbai_data[south_mumbai_data.Neighborhood != 'Malabar Hill']
print("aft:",south_mumbai_data.shape)

bef: (30, 5)
aft: (29, 5)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


#### **Remove Redundant Data**

In [60]:
south_mumbai_venues_worli = south_mumbai_venues[south_mumbai_venues['Neighborhood'] == 'Worli']
south_mumbai_venues_worli = south_mumbai_venues_worli[south_mumbai_venues['Venue Category'] == 'Scenic Lookout']
                                          
south_mumbai_venues_worli.head(15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
400,573,Worli,19.011696,72.81807,Worli Sea Face,19.009216,72.815022,Scenic Lookout
403,576,Worli,19.011696,72.81807,Worli Drive,19.012063,72.815610,Scenic Lookout
413,587,Worli,19.011696,72.81807,"Century Bazaar, Worli",19.012736,72.823949,Scenic Lookout


In [61]:
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Worli']

test_df = south_mumbai_venues
venue_list = ['Worli Drive','Century Bazaar, Worli','Sea Corner']
print("bef:",test_df.shape)
test_df = test_df[test_df['Venue'].isin(venue_list) == False]
print("aft:",test_df.shape)
south_mumbai_venues = test_df
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Worli']



bef: (866, 8)
aft: (864, 8)


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
400,573,Worli,19.011696,72.81807,Worli Sea Face,19.009216,72.815022,Scenic Lookout
401,574,Worli,19.011696,72.81807,Slink & Bardot,19.017170,72.817532,French Restaurant
402,575,Worli,19.011696,72.81807,Starbucks Coffee: A Tata Alliance,19.012851,72.823886,Coffee Shop
404,577,Worli,19.011696,72.81807,Cafe Nemo,19.017172,72.817548,Café
405,578,Worli,19.011696,72.81807,Cool Chef Café,19.017002,72.817544,Café
406,579,Worli,19.011696,72.81807,Indiabulls.parel.,19.011488,72.820471,Paper / Office Supplies Store
407,580,Worli,19.011696,72.81807,Google India Ltd,19.011501,72.820469,Fitness/Gym/Spa
408,581,Worli,19.011696,72.81807,National sports club of India,19.014701,72.817081,Sports Club
409,583,Worli,19.011696,72.81807,Sandwich Guy,19.011256,72.821646,Sandwich Place
410,584,Worli,19.011696,72.81807,Juice Joint,19.010453,72.821536,Juice Bar


### **Correction of some categories**

In [62]:
#df1.loc[df1['stream'] == 2, 'feat'] = 10

test_df = south_mumbai_venues

test_df.loc[test_df['Venue'] == 'Boat ride, gateway of india', 'Venue Category'] = 'Boat ride'

test_df.iloc[70:72]
#south_mumbai_venues = test_df

# Dropped Harbor / Marina from Masjid Bandar
test_df = south_mumbai_venues
venue_cat_list = ['Harbor / Marina']
print("bef:",test_df.shape)
test_df = test_df[test_df['Venue Category'].isin(venue_cat_list) == False]
print("aft:",test_df.shape)

south_mumbai_venues = test_df
south_mumbai_venues.loc[south_mumbai_venues['Neighborhood'] == 'Masjid Bandar']



bef: (864, 8)
aft: (859, 8)


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
310,412,Masjid Bandar,18.953599,72.838126,Patel Restaurant,18.949798,72.834655,Restaurant
311,414,Masjid Bandar,18.953599,72.838126,Chinese-n-Grill,18.954864,72.833804,Chinese Restaurant
312,415,Masjid Bandar,18.953599,72.838126,Elphinestone Hotel,18.948737,72.840571,Hotel
313,418,Masjid Bandar,18.953599,72.838126,Masjid Bunder Market,18.953300,72.837081,Convenience Store
314,419,Masjid Bandar,18.953599,72.838126,Jeetu Sandwich,18.956031,72.837381,Sandwich Place
315,420,Masjid Bandar,18.953599,72.838126,Abu Baba,18.955005,72.834744,Smoke Shop
317,423,Masjid Bandar,18.953599,72.838126,Le Mill,18.957379,72.840742,Furniture / Home Store
318,425,Masjid Bandar,18.953599,72.838126,Green Fields Restaurant,18.957929,72.835892,Smoke Shop
319,426,Masjid Bandar,18.953599,72.838126,Simla,18.957929,72.835817,Café
321,428,Masjid Bandar,18.953599,72.838126,Musafir Khana,18.948624,72.840431,Convenience Store


# **Iteration: 1**

### **Remove certain categories and do an iteration.**


In [64]:
venue_category = ['Indian Restaurant','Bar/Pub','Coffee Shop','Restaurant','Fast Food Restaurant','Train/Metro/Bus','Café']
test_df = south_mumbai_venues
test_df = test_df[test_df['Venue Category'].isin(venue_category) == False]
test_df.shape
south_mumbai_venues = test_df
south_mumbai_venues.shape

(619, 8)

# **Iteration: 2**

### **Remove some more categories and do an iteration.**

In [0]:
test_df.to_csv("groupby_nbhood.csv")

# **Checkpoint:1:**

### **Backup : south_mumbai_venues**

In [0]:
south_mumbai_venues_bkp1 = south_mumbai_venues
south_mumbai_venues_bkp2 = south_mumbai_venues
south_mumbai_venues_bkp3 = south_mumbai_venues



In [0]:
south_mumbai_venues.to_csv("south_mumbai_venues_clean.csv")

In [67]:
print("After all cleanup:Total venues:",south_mumbai_venues.shape[0])
print("After all cleanup:Total neigborhoods:",neighborhoods.shape[0])


After all cleanup:Total venues: 619
After all cleanup:Total neigborhoods: 39


### **Data Cleansing ends here.**

## **Analyze Each Neighborhood of Mumbai.**

In [68]:
# one hot encoding
south_mumbai_onehot = pd.get_dummies(south_mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
south_mumbai_onehot['Neighborhood'] = south_mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [south_mumbai_onehot.columns[-1]] + list(south_mumbai_onehot.columns[:-1])
south_mumbai_onehot = south_mumbai_onehot[fixed_columns]

south_mumbai_onehot.shape


(619, 129)

In [69]:
#And let's examine the new dataframe size.
south_mumbai_onehot.shape  

(619, 129)

# **Group rows by neighborhood.**

### **Take mean of the frequency of occurrence of each category.**

In [75]:
south_mumbai_grouped = south_mumbai_onehot.groupby('Neighborhood').mean().reset_index()
print("This matrix has {} rows and {} columns".format(south_mumbai_grouped.shape[0],south_mumbai_grouped.shape[1]))
south_mumbai_grouped.head(10)

This matrix has 29 rows and 129 columns


,Neighborhood,Accessories Store,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Boat or Ferry,Boat ride,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,College Academic Building,College Auditorium,College Technology Building,Comedy Club,Comfort Food Restaurant,Convenience Store,Creperie,Cupcake Shop,...,Monument / Landmark,Mughlai Restaurant,Music Store,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park/Garden,Parsi Restaurant,Pharmacy,Pizza Place,Platform,Plaza,Pool,Racetrack,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Spanish Restaurant,Sports,Sports Club,Steakhouse,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater\Multiplex,Toy / Game Store,Vegetarian / Vegan Restaurant,Women's Store,Zoo
0,Andheri,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.125000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.125000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Apollo Bandar,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.055556,0.000000,0.000000,0.027778,0.0,0.000000,0.0,0.0,0.000000,0.0,0.083333,0.027778,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.055556,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.027778,0.055556,0.0,0.0,0.027778,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.027778,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.027778,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.027778,0.000000,0.000000,0.0
2,Bandra West,0.0,0.000000,0.0,0.0,0.044118,0.0,0.0,0.058824,0.014706,0.014706,0.088235,0.0,0.000000,0.0,0.0,0.014706,0.0,0.000000,0.000000,0.029412,0.000000,0.014706,0.000000,0.0,0.014706,0.0,0.0,0.0,0.058824,0.014706,0.0,0.000000,0.014706,0.0,0.014706,0.0,0.000000,0.0,0.0,...,0.000000,0.014706,0.0,0.0,0.014706,0.000000,0.0,0.000000,0.0,0.000000,0.014706,0.000000,0.0,0.058824,0.000000,0.000000,0.0,0.0,0.014706,0.000000,0.014706,0.0,0.000000,0.044118,0.0,0.000000,0.014706,0.029412,0.014706,0.000000,0.0,0.014706,0.000000,0.0,0.000000,0.014706,0.000000,0.014706,0.014706,0.0
3,Bhuleshwar,0.0,0.000000,0.0,0.0,0.111111,0.0,0.0,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.111111,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Borivali,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.045455,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.045455,0.0,0.0,0.0,0.090909,0.136364,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.045455,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.045455,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.090909,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.045455,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.045455,0.0

In [73]:
south_mumbai_grouped.shape

(29, 129)

In [0]:
south_mumbai_grouped.to_csv("south_mumbai_grouped.csv")
# backup
south_mumbai_grouped_bkp = south_mumbai_grouped

In [0]:
## Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in south_mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = south_mumbai_grouped[south_mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Andheri----
            venue  freq
0  Shop & Service   0.1
1        Platform   0.1
2  Sandwich Place   0.1
3    Burger Joint   0.1
4     Snack Place   0.1


----Apollo Bandar----
                venue  freq
0               Hotel  0.19
1       Boat or Ferry  0.09
2  Mughlai Restaurant  0.06
3  Chinese Restaurant  0.06
4     Fitness/Gym/Spa  0.06


----Bandra West----
                venue  freq
0              Bakery  0.09
1         Pizza Place  0.06
2  Chinese Restaurant  0.06
3    Asian Restaurant  0.06
4     Fitness/Gym/Spa  0.04


----Bhuleshwar----
            venue  freq
0  Ice Cream Shop   0.1
1       BBQ Joint   0.1
2       Juice Bar   0.1
3     Snack Place   0.1
4    Dessert Shop   0.1


----Borivali----
                venue  freq
0      Ice Cream Shop  0.14
1  Chinese Restaurant  0.09
2      Sandwich Place  0.09
3           Juice Bar  0.09
4         Snack Place  0.09


----Chembur----
                           venue  freq
0             Seafood Restaurant  0.13
1         

## **Let's put that into a pandas dataframe.**

## **Let's write a function to sort the venues in descending order.**

In [0]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# **Display the top 10 venues for each neighborhood.**

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = south_mumbai_grouped['Neighborhood']

for ind in np.arange(south_mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(south_mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Andheri,Golf Course,Food Court,Dessert Shop,Snack Place,Bakery,Sandwich Place,Burger Joint,Food Truck,Cupcake Shop,Creperie
1,Apollo Bandar,Hotel,Boat or Ferry,Mughlai Restaurant,Chinese Restaurant,Fitness/Gym/Spa,Asian Restaurant,Sculpture Garden,Mediterranean Restaurant,Lounge,Juice Bar
2,Bandra West,Bakery,Asian Restaurant,Chinese Restaurant,Pizza Place,Arcade,Seafood Restaurant,Dessert Shop,Fitness/Gym/Spa,Snack Place,Event Space
3,Bhuleshwar,Dessert Shop,Indian Sweet Shop,Arcade,Chinese Restaurant,Ice Cream Shop,BBQ Joint,Juice Bar,Snack Place,Zoo,Dhaba
4,Borivali,Ice Cream Shop,Clothing Store,Chinese Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Theater\Multiplex,Snack Place,Juice Bar,Pizza Place,Department Store


In [0]:
# save to csv file
neighborhoods_venues_sorted.to_csv("neighborhoods_venues_sorted.csv")
#backup 
neighborhoods_venues_sorted_bkp = neighborhoods_venues_sorted

In [84]:
neighborhoods_venues_sorted.shape

(29, 11)

# **Cluster Neighborhood**

## **Run k-means. Adjusting number of clusters to optimum.**

In [85]:
kclusters = 5
south_mumbai_grouped_clustering = south_mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(south_mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_[0:40] 


array([3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 2, 4, 3, 3, 2, 3, 0, 3, 3, 3,
       3, 2, 3, 3, 2, 3, 1], dtype=int32)

### **Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.**

## **Add clustering labels.**

In [0]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [87]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Andheri,Golf Course,Food Court,Dessert Shop,Snack Place,Bakery,Sandwich Place,Burger Joint,Food Truck,Cupcake Shop,Creperie
1,2,Apollo Bandar,Hotel,Boat or Ferry,Mughlai Restaurant,Chinese Restaurant,Fitness/Gym/Spa,Asian Restaurant,Sculpture Garden,Mediterranean Restaurant,Lounge,Juice Bar
2,3,Bandra West,Bakery,Asian Restaurant,Chinese Restaurant,Pizza Place,Arcade,Seafood Restaurant,Dessert Shop,Fitness/Gym/Spa,Snack Place,Event Space
3,3,Bhuleshwar,Dessert Shop,Indian Sweet Shop,Arcade,Chinese Restaurant,Ice Cream Shop,BBQ Joint,Juice Bar,Snack Place,Zoo,Dhaba
4,3,Borivali,Ice Cream Shop,Clothing Store,Chinese Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Theater\Multiplex,Snack Place,Juice Bar,Pizza Place,Department Store


## **Now merge both dataframes to include borough.**

In [89]:
south_mumbai_data.head(2)
#south_mumbai_data.shape
neighborhoods_venues_sorted.shape

(29, 12)

In [90]:
south_mumbai_merged = south_mumbai_data

# merge two dataframes
south_mumbai_merged = south_mumbai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

south_mumbai_merged.head() 


,index,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Colaba,South Mumbai,18.915091,72.825969,2,Fitness/Gym/Spa,Diner,Hotel,Chinese Restaurant,Donut Shop,Japanese Restaurant,Italian Restaurant,Dessert Shop,Pizza Place,Farmers Market
1,1,Apollo Bandar,South Mumbai,18.918375,72.831443,2,Hotel,Boat or Ferry,Mughlai Restaurant,Chinese Restaurant,Fitness/Gym/Spa,Asian Restaurant,Sculpture Garden,Mediterranean Restaurant,Lounge,Juice Bar
2,2,Fort,South Mumbai,18.933266,72.834515,3,Chinese Restaurant,Seafood Restaurant,Dessert Shop,Irani Cafe,Parsi Restaurant,Plaza,Lounge,Clothing Store,Mediterranean Restaurant,Men's Store
3,3,Churchgate,South Mumbai,18.935957,72.827340,3,Sports,Ice Cream Shop,Hotel,Italian Restaurant,Theater\Multiplex,Pizza Place,Bakery,Japanese Restaurant,Dessert Shop,Sandwich Place
4,4,Nariman Point,South Mumbai,18.925951,72.823208,3,Theater\Multiplex,Italian Restaurant,Hotel,Mediterranean Restaurant,Chaat Place,Sports,Snack Place,Diner,Bookstore,Scenic Lookout


In [0]:
# save to csv file
south_mumbai_merged.to_csv("south_mumbai_merged.csv")
# backup
south_mumbai_merged_bkp = south_mumbai_merged


In [0]:
south_mumbai_merged.tail()

,index,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,33,Mulund West,East Mumbai,19.171972,72.951196,0,Dessert Shop,Theater,Music Venue,Snack Place,Ice Cream Shop,Shopping Mall,Flea Market,Sports,Creperie,Cupcake Shop
34,34,Powai,North Mumbai,19.118720,72.907348,3,Ice Cream Shop,Fitness/Gym/Spa,Chinese Restaurant,Bakery,Pizza Place,Italian Restaurant,Dessert Shop,Donut Shop,Lounge,Tea Room
35,35,Jogeshwari,North Mumbai,19.134899,72.848820,0,Asian Restaurant,Chinese Restaurant,Mughlai Restaurant,Pizza Place,Ice Cream Shop,Department Store,Diner,Dim Sum Restaurant,Dhaba,Dessert Shop
37,37,Borivali,North Mumbai,19.229068,72.857363,0,Ice Cream Shop,Clothing Store,Chinese Restaurant,Juice Bar,Sandwich Place,Snack Place,Lounge,Vegetarian / Vegan Restaurant,Theater,Food Truck
38,38,Mira Road,North Mumbai,19.187896,72.836596,3,Clothing Store,Chinese Restaurant,Fitness/Gym/Spa,Shopping Mall,Diner,Pizza Place,Donut Shop,Dim Sum Restaurant,Burger Joint,Burrito Place


In [0]:
south_mumbai_merged.to_csv("south_mumbai_merged.csv")

### check **Cluster Labels** column has been added to dataframe. 

In [99]:
print("fdffd")

fdffd


### **Clusters given name**

Cluster Labels 0 : Convenience Store,Smoke Shop

Cluster Label  1 : Scenic Lookout,Sports Club

Cluster Label  2 : Hotel,Fitness/Gym/Spa,Chinese Restaurant

Cluster Label  3 : Theater\Multiplex,Ice Cream Shop,Seafood Restaurant

Cluster Label  4 : American Restaurant,Market


Following most suitable names based on top venue categories was given to each cluster as follows.


# **Visualize Clusters**

In [94]:
latitude = mumbai_latitude
longitude = mumbai_longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(south_mumbai_merged['Latitude'], south_mumbai_merged['Longitude'], south_mumbai_merged['Neighborhood'], south_mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# **Second Iteration** 

The approach this time is to just pick tourist spots like museum, aquarium, monumnet, beaches,zoo,boat ride,plaza etc. and run analysis on that.

In [0]:
#south_mumbai_venues = south_mumbai_venues_it1

In [114]:
#backup iteration 1
#south_mumbai_venues_it1 = south_mumbai_venues

# club certain categories first
south_mumbai_venues.replace(to_replace =['Boat ride'],value = 'Boat or Ferry', inplace=True)
south_mumbai_venues.replace(to_replace =['Sculpture Garden'],value = 'Monument / Landmark', inplace=True)
south_mumbai_venues.replace(to_replace =['Other Great Outdoors'],value = 'Scenic Lookout', inplace=True)
south_mumbai_venues.replace(to_replace =['Jetty - Gateway of India'],value = 'Scenic Lookout', inplace=True)



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [116]:
#'Garden','Park/Garden','Plaza'
south_mumbai_venues.shape #(609, 8)
test_df = south_mumbai_venues_bkp1
venue_cat_list = ['Monument / Landmark','Boat or Ferry','Sculpture Garden' \
                 ,'History Museum','Other Great Outdoors','Scenic Lookout','Aquarium' \
                  ,'Zoo','Racetrack','Beach','Boat ride'
                  ]

print("bef:",test_df.shape)
test_df = test_df[test_df['Venue Category'].isin(venue_cat_list) == True]
print("aft:",test_df.shape) # (22, 8)
test_df.head()
south_mumbai_venues = test_df
print("aft:" , south_mumbai_venues.shape)

south_mumbai_venues.head()


                  

bef: (619, 8)
aft: (20, 8)
aft: (20, 8)


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
45,56,Apollo Bandar,18.918375,72.831443,Gateway of India,18.921856,72.834877,Monument / Landmark
77,103,Apollo Bandar,18.918375,72.831443,APM Deck @ Gateway of India,18.922174,72.833947,Boat or Ferry
78,104,Apollo Bandar,18.918375,72.831443,"Boat ride, gateway of india",18.921847,72.834652,Boat or Ferry
80,106,Apollo Bandar,18.918375,72.831443,Chhatrapati Shivaji Monument,18.922713,72.834172,Monument / Landmark
81,107,Apollo Bandar,18.918375,72.831443,Jetty - Gateway of India,18.922558,72.834640,Boat or Ferry


In [0]:
#backup iteration 1
south_mumbai_venues_it_2 = south_mumbai_venues

# to file
south_mumbai_venues_it_2.to_csv("south_mumbai_venues_it_2.csv")

In [108]:
south_mumbai_venues_it_2.shape

(24, 8)

In [119]:
# one hot encoding
south_mumbai_onehot = pd.get_dummies(south_mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
south_mumbai_onehot['Neighborhood'] = south_mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [south_mumbai_onehot.columns[-1]] + list(south_mumbai_onehot.columns[:-1])
south_mumbai_onehot = south_mumbai_onehot[fixed_columns]

south_mumbai_onehot.shape


(20, 9)

In [120]:
south_mumbai_grouped = south_mumbai_onehot.groupby('Neighborhood').mean().reset_index()
south_mumbai_grouped.head(1)

,Neighborhood,Aquarium,Beach,Boat or Ferry,History Museum,Monument / Landmark,Racetrack,Scenic Lookout,Zoo
0,Apollo Bandar,0.0,0.0,0.666667,0.0,0.333333,0.0,0.0,0.0


In [0]:
south_mumbai_grouped_it_2 = south_mumbai_grouped
south_mumbai_grouped_it_2.to_csv("south_mumbai_grouped_it_2.csv")

In [122]:
## Let's print each neighborhood along with the top 2 most common venues

num_top_venues = 2

for hood in south_mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = south_mumbai_grouped[south_mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Apollo Bandar----
                 venue  freq
0        Boat or Ferry  0.67
1  Monument / Landmark  0.33


----Churchgate----
            venue  freq
0           Beach   0.5
1  Scenic Lookout   0.5


----Fort----
                 venue  freq
0       History Museum   0.5
1  Monument / Landmark   0.5


----Juhu----
      venue  freq
0     Beach   1.0
1  Aquarium   0.0


----Mahalaxmi----
       venue  freq
0  Racetrack   1.0
1   Aquarium   0.0


----Marine Lines----
            venue  freq
0        Aquarium   0.5
1  Scenic Lookout   0.5


----Nagpada----
            venue  freq
0  History Museum   0.5
1             Zoo   0.5


----Nariman Point----
            venue  freq
0  Scenic Lookout   1.0
1        Aquarium   0.0


----Powai----
            venue  freq
0  Scenic Lookout   1.0
1        Aquarium   0.0


----Walkeshwar----
            venue  freq
0  History Museum   1.0
1        Aquarium   0.0


----Worli----
            venue  freq
0  Scenic Lookout   1.0
1        Aquarium   0.0


In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### **Take top 2**

In [124]:

num_top_venues = 2

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators
[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = south_mumbai_grouped['Neighborhood']

for ind in np.arange(south_mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(south_mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue
0,Apollo Bandar,Boat or Ferry,Monument / Landmark
1,Churchgate,Scenic Lookout,Beach
2,Fort,Monument / Landmark,History Museum
3,Juhu,Beach,Zoo
4,Mahalaxmi,Racetrack,Zoo


In [0]:
neighborhoods_venues_sorted_it_2 = neighborhoods_venues_sorted
neighborhoods_venues_sorted_it_2.to_csv("neighborhoods_venues_sorted_it_2.csv")

In [126]:
kclusters = 10
south_mumbai_grouped_clustering = south_mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(south_mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_[0:40] 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  """


array([4, 7, 1, 0, 3, 8, 5, 2, 2, 6, 2], dtype=int32)

In [0]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [0]:
south_mumbai_merged = south_mumbai_data

south_mumbai_merged = pd.merge(south_mumbai_merged, neighborhoods_venues_sorted, on='Neighborhood', how='inner')






In [0]:
south_mumbai_merged.to_csv("south_mumbai_merged_It2.csv")
south_mumbai_merged_it_2 = south_mumbai_merged

In [0]:
### Old Code :
#south_mumbai_merged = south_mumbai_data
### merge two dataframes
#south_mumbai_merged = south_mumbai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
####pd.merge(df1, df2, on='Customer_id', how='inner')
#south_mumbai_merged.head()

In [130]:
south_mumbai_merged.head(15)

,index,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue
0,1,Apollo Bandar,South Mumbai,18.918375,72.831443,4,Boat or Ferry,Monument / Landmark
1,2,Fort,South Mumbai,18.933266,72.834515,1,Monument / Landmark,History Museum
2,3,Churchgate,South Mumbai,18.935957,72.827340,7,Scenic Lookout,Beach
3,4,Nariman Point,South Mumbai,18.925951,72.823208,2,Scenic Lookout,Zoo
4,5,Marine Lines,South Mumbai,18.945670,72.823781,8,Scenic Lookout,Aquarium
5,6,Walkeshwar,South Mumbai,18.955343,72.807947,6,History Museum,Zoo
6,13,Nagpada,South Mumbai,18.968178,72.828601,5,Zoo,History Museum
7,19,Mahalaxmi,South Mumbai,18.982568,72.824160,3,Racetrack,Zoo
8,20,Worli,South Mumbai,19.011696,72.818070,2,Scenic Lookout,Zoo
9,23,Juhu,West Mumbai,19.107021,72.827528,0,Beach,Zoo


In [0]:
south_mumbai_merged.to_csv("south_mumbai_merged_It2.csv")

In [131]:
atitude = mumbai_latitude
longitude = mumbai_longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(south_mumbai_merged['Latitude'], south_mumbai_merged['Longitude'], south_mumbai_merged['Neighborhood'], south_mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
